In [1]:
import time
import numpy as np
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [2]:
opt = webdriver.ChromeOptions()
opt.add_argument("--incognito")
opt.add_argument("disable-infobars")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-gpu")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument("--headless=new")

driver = webdriver.Chrome(options=opt)
driver.get("https://safebooru.org/index.php?page=tags&s=list&pid=0")
wait = WebDriverWait(driver, 20)

In [3]:
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "pagination")))

<selenium.webdriver.remote.webelement.WebElement (session="6d76736378373ff822582dfa60db5c0f", element="f.84BFF2758A8CBFB1B34109B05DD99362.d.33534FD9366C16D72D64C8F3746AB6E5.e.7")>

In [4]:
#page = driver.page_source
#soup = BeautifulSoup(page, "lxml")

In [5]:
#tag_table = soup.find(class_="highlightable")

In [9]:
#tags = tag_table.find_all("span", class_=["tag-type-copyright", "tag-type-artist", "tag-type-general", "tag-type-character", "tag-type-metadata"])#href=True
#tags = tag_table.find_all("a")

In [34]:
#one = tags[0]
#one["class"][0][9:]#[21:-1]

'general'

In [8]:
#button_location = driver.find_element(By.CLASS_NAME, "pagination")
#button = button_location.find_element(By.XPATH, '//a[@alt="next"]')
#button.click()

In [9]:
#button_location.text

In [10]:
#soup.find(alt="next") != None

In [4]:
tags = []
scrape = True

while scrape:
    try:
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "pagination")))
        page = driver.page_source
        soup = BeautifulSoup(page, "lxml")
        tag_table = soup.find(class_="highlightable")
        tags += tag_table.find_all("span", class_=["tag-type-copyright", "tag-type-artist", "tag-type-general", "tag-type-character", "tag-type-metadata"])
    
        if soup.find(alt="next") != None:
            button_location = driver.find_element(By.CLASS_NAME, "pagination")
            button = button_location.find_element(By.XPATH, '//a[@alt="next"]')
            button.click()
        else:
            scrape = False
    except TimeoutException:
        driver.refresh()

In [5]:
len(tags)

659523

In [6]:
len(set(tags))

658879

In [7]:
driver.close()

In [8]:
nd_tags = list(set(tags))

In [9]:
url = 'https://safebooru.org'

In [10]:
nd_tags[0:5]

[<span class="tag-type-general"><a href="index.php?page=post&amp;s=list&amp;tags=sleepymausu">sleepymausu</a></span>,
 <span class="tag-type-artist"><a href="index.php?page=post&amp;s=list&amp;tags=hiro_%28725611%29">hiro_(725611)</a></span>,
 <span class="tag-type-general"><a href="index.php?page=post&amp;s=list&amp;tags=ind_%28121%29">ind_(121)</a></span>,
 <span class="tag-type-artist"><a href="index.php?page=post&amp;s=list&amp;tags=shouri_%28harpy_killer%29">shouri_(harpy_killer)</a></span>,
 <span class="tag-type-general"><a href="index.php?page=post&amp;s=list&amp;tags=jun_%28pokemon%29_%28cosplay%29">jun_(pokemon)_(cosplay)</a></span>]

In [54]:
main_cat = []
names = []
links = []
for i in nd_tags:
    main_cat.append(i["class"][0][9:])
    get_ = i.find('a')['href'].strip().split('/')[-1]
    link = "{}/{}".format(url, get_)
    names.append(i.text)
    links.append(link)

In [55]:
data = {"Main_Category": main_cat, "Name": names, "Link": links}

In [56]:
df = pd.DataFrame.from_dict(data)
df

,Main_Category,Name,Link
0,general,sleepymausu,https://safebooru.org/index.php?page=post&s=li...
1,artist,hiro_(725611),https://safebooru.org/index.php?page=post&s=li...
2,general,ind_(121),https://safebooru.org/index.php?page=post&s=li...
3,artist,shouri_(harpy_killer),https://safebooru.org/index.php?page=post&s=li...
4,general,jun_(pokemon)_(cosplay),https://safebooru.org/index.php?page=post&s=li...
...,...,...,...
658874,general,obreasts,https://safebooru.org/index.php?page=post&s=li...
658875,character,darcmon,https://safebooru.org/index.php?page=post&s=li...
658876,character,phecda,https://safebooru.org/index.php?page=post&s=li...
658877,general,wster,https://safebooru.org/index.php?page=post&s=li...


In [57]:
df.to_csv("../Datasets/safebooru_tags_links_2.csv", index=False)